In [14]:
import matplotlib.pyplot as plt
import numpy as np
import scipy
import cv2
import skimage.io as skio
import skimage as sk
from scipy.ndimage import distance_transform_edt as edt

# Part A.2

In [2]:
def get_image_points(im, num_points=-1, timeout=0):
    # Remember to do %matplotlib qt before running this function.
    plt.imshow(im)
    points = plt.ginput(n=num_points, timeout=timeout)
    plt.close('all')
    return points

In [3]:
def get_ls_inputs(im1_pts, im2_pts):
    if len(im1_pts) != len(im2_pts):
        print("point lists are of different lengths.")
        return
    A = []
    b = []
    for i in range(len(im1_pts)):
        y1, x1 = im1_pts[i]
        y2, x2 = im2_pts[i]
        A.append(np.array([x1, y1, 1, 0, 0, 0, -x1 * x2, -y1 * x2]))
        b.append(x2)
        A.append(np.array([0, 0, 0, x1, y1, 1, -x1 * y2, -y1 * y2]))
        b.append(y2)
    A = np.vstack(tuple(A))
    b = np.array(b)
    return A, b

def computeH(im1_pts, im2_pts):
    A, b = get_ls_inputs(im1_pts, im2_pts)
    h, _, _, _ = np.linalg.lstsq(A, b, rcond=None)
    return np.array([[h[0], h[1], h[2]],
                     [h[3], h[4], h[5]],
                     [h[6], h[7], 1]])

In [4]:
%matplotlib qt

In [7]:
im1_pts = get_image_points("data/A.1/set1/1.jpg")

In [8]:
im2_pts = get_image_points("data/A.1/set1/2.jpg")

In [16]:
A, b = get_ls_inputs(im1_pts, im2_pts)
H = computeH(im1_pts, im2_pts)

# Part A.3

In [83]:
def get_xy(homogenous_coords):
    return int(homogenous_coords[0] / homogenous_coords[2]), homogenous_coords[1] / homogenous_coords[2]
def get_xy_int(homogenous_coords):
    return int(homogenous_coords[0] / homogenous_coords[2]), int(homogenous_coords[1] / homogenous_coords[2])
def get_binary_mask(im, H):
    channel_height, channel_width = im.shape[:2]
    x1, y1 = get_xy_int(H @ np.array([0, 0, 1]))
    x2, y2 = get_xy_int(H @ np.array([channel_height, 0, 1]))
    x3, y3 = get_xy_int(H @ np.array([0, channel_width, 1]))
    x4, y4 = get_xy_int(H @ np.array([channel_height, channel_width, 1]))
    x_init = min(x1, x2, x3, x4)
    y_init = min(y1, y2, y3, y4)
    x_max = max(x1, x2, x3, x4)
    y_max = max(y1, y2, y3, y4)
    dx, dy = x_max - x_init, y_max - y_init
    res = [[0.0] * (dy + 1) for _ in range(dx + 1)]

    H_inv = np.linalg.inv(H)

    print(dx, dy)

    for x in range(dx + 1):
        for y in range(dy + 1):
            new_homogenous_coords = np.array([x_init + x, y_init + y, 1])
            original_homogenous_coords = H_inv @ new_homogenous_coords
            original_x, original_y = get_xy_int(original_homogenous_coords)
            if original_x < 0 or original_x >= channel_height:
                continue
            if original_y < 0 or original_y >= channel_width:
                continue
            res[x][y] = 1.0
    
    return np.array(res), x_init, y_init

In [ ]:
def warp_image_one_channel_nn(channel, H):
    channel_height, channel_width = channel.shape
    x1, y1 = get_xy_int(H @ np.array([0, 0, 1]))
    x2, y2 = get_xy_int(H @ np.array([channel_height, 0, 1]))
    x3, y3 = get_xy_int(H @ np.array([0, channel_width, 1]))
    x4, y4 = get_xy_int(H @ np.array([channel_height, channel_width, 1]))
    x_init = min(x1, x2, x3, x4)
    y_init = min(y1, y2, y3, y4)
    x_max = max(x1, x2, x3, x4)
    y_max = max(y1, y2, y3, y4)
    dx, dy = x_max - x_init, y_max - y_init
    res = [[0.0] * (dy + 1) for _ in range(dx + 1)]

    H_inv = np.linalg.inv(H)

    print(dx, dy)

    for x in range(dx + 1):
        for y in range(dy + 1):
            new_homogenous_coords = np.array([x_init + x, y_init + y, 1])
            original_homogenous_coords = H_inv @ new_homogenous_coords
            original_x, original_y = get_xy_int(original_homogenous_coords)
            if original_x < 0 or original_x >= channel_height:
                continue
            if original_y < 0 or original_y >= channel_width:
                continue
            res[x][y] = channel[original_x][original_y]
    
    return np.array(res), x_init, y_init

def warp_image_helper_nn(im, H):
    res = []
    for channel_num in range(im.shape[-1]):
        channel_res, offset_x, offset_y = warp_image_one_channel_nn(im[:, :, channel_num], H)
        res.append(channel_res)
    res = np.dstack(tuple(res))
    return res, offset_x, offset_y

def warpImageNearestNeighbor(im, H):
    return warp_image_helper_nn(im, H)[0]

In [84]:
def warp_image_one_channel_bilinear(channel, H):
    channel_height, channel_width = channel.shape
    x1, y1 = get_xy_int(H @ np.array([0, 0, 1]))
    x2, y2 = get_xy_int(H @ np.array([channel_height, 0, 1]))
    x3, y3 = get_xy_int(H @ np.array([0, channel_width, 1]))
    x4, y4 = get_xy_int(H @ np.array([channel_height, channel_width, 1]))
    x_init = min(x1, x2, x3, x4)
    y_init = min(y1, y2, y3, y4)
    x_max = max(x1, x2, x3, x4)
    y_max = max(y1, y2, y3, y4)
    dx, dy = x_max - x_init, y_max - y_init
    res = [[0.0] * (dy + 1) for _ in range(dx + 1)]

    H_inv = np.linalg.inv(H)

    print(dx, dy)

    for x in range(dx + 1):
        for y in range(dy + 1):
            new_homogenous_coords = np.array([x_init + x, y_init + y, 1])
            original_homogenous_coords = H_inv @ new_homogenous_coords
            original_x, original_y = get_xy(original_homogenous_coords)
            if original_x <= 0.0 or original_x >= channel_height - 1.0:
                continue
            if original_y <= 0.0 or original_y >= channel_width - 1.0:
                continue
            x_smaller = int(original_x)
            x_bigger = int(original_x + 1)
            y_smaller = int(original_y)
            y_bigger = int(original_y + 1)
            topleft = channel[x_smaller][y_smaller] * (x_bigger - original_x) * (y_bigger - original_y)
            topright = channel[x_smaller][y_bigger] * (x_bigger - original_x) * (original_y - y_smaller)
            bottomleft = channel[x_bigger][y_smaller] * (original_x - x_smaller) * (y_bigger - original_y)
            bottomright = channel[x_bigger][y_bigger] * (original_x - x_smaller) * (original_y - y_smaller)
            res[x][y] = topleft + topright + bottomleft + bottomright
    
    return np.array(res), x_init, y_init

def warp_image_helper_bilinear(im, H):
    res = []
    for channel_num in range(im.shape[-1]):
        channel_res, offset_x, offset_y = warp_image_one_channel_bilinear(im[:, :, channel_num], H)
        res.append(channel_res)
    res = np.dstack(tuple(res))
    return res, offset_x, offset_y

def warpImageBilinear(im, H):
    return warp_image_helper_bilinear(im, H)[0]

In [426]:
im = skio.imread("data/A.3/3.jpg")
im_float = (im / 255).astype(np.float32)
im_float = im_float[:, :, :3]

In [428]:
im1_pts = get_image_points(im_float)

In [417]:
im1_pts

[(np.float64(710.6401869158881), np.float64(573.2824978759554)),
 (np.float64(1516.5875106202207), np.float64(859.2638062871706)),
 (np.float64(600.1474086661005), np.float64(2094.183092608326)),
 (np.float64(1451.591758708581), np.float64(2159.178844519966))]

In [429]:
square_points = [(0.0, 0.0), (200.0, 0.0), (0.0, 300.0), (200.0, 300.0)]

In [430]:
H = computeH(im1_pts, square_points)

In [434]:
res, offset_x, offset_y = warp_image_helper_bilinear(im_float, H)

1036 713
1036 713
1036 713


In [435]:
skio.imshow(res)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/4211006476.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(res)


In [436]:
skio.imsave("data/A.3/3_bilinear.jpg", (res * 255).astype(np.uint8))

In [95]:
im = skio.imread("data/A.3/2.jpg")
im_float = (im / 255).astype(np.float32)

# skio read portrait image as landscape, so I'm rotating it back.
im_float = np.rot90(im_float, k=-1, axes=(0, 1))

In [109]:
im1_pts = get_image_points(im_float)

In [110]:
im1_pts

[(np.float64(373.61554800339854), np.float64(1544.4923534409509)),
 (np.float64(3935.3827527612557), np.float64(1509.8279524214104)),
 (np.float64(1240.2255734919281), np.float64(4334.976635514018)),
 (np.float64(3112.103228547153), np.float64(4343.6427357689045))]

In [111]:
rectangle_points = [(0.0, 0.0), (400.0, 0.0), (0.0, 600.0), (400.0, 600.0)]

In [112]:
H = computeH(im1_pts, rectangle_points)

In [116]:
res2, offset_x2, offset_y2 = warp_image_helper_bilinear(im_float, H)

1768 1639
1768 1639
1768 1639


In [117]:
skio.imshow(res2)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/2359979467.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(res2)


In [118]:
skio.imsave("data/A.3/2_bilinear.jpg", (res2 * 255).astype(np.uint8))

# Part 8.4

In [437]:
im1 = skio.imread("data/A.1/set5/1.jpg")
im1_float = (im1 / 255).astype(np.float32)
im1_float = cv2.resize(im1_float, None, fx=0.3, fy=0.3)
im2 = skio.imread("data/A.1/set5/2.jpg")
im2_float = (im2 / 255).astype(np.float32)
im2_float = cv2.resize(im2_float, None, fx=0.3, fy=0.3)
im3 = skio.imread("data/A.1/set5/3.jpg")
im3_float = (im3 / 255).astype(np.float32)
im3_float = cv2.resize(im3_float, None, fx=0.3, fy=0.3)

#### Put all three images on the same mosaic

In [120]:
# OPTIONAL: use saved points.
im1_pts = [(np.float64(1484.2612574341551), np.float64(536.1773880325283)),
 (np.float64(1665.5711858235227), np.float64(532.2782497875957)),
 (np.float64(1482.3116883116886), np.float64(758.4282679936887)),
 (np.float64(1663.6216167010566), np.float64(762.3274062386213)),
 (np.float64(1314.6487437795854), np.float64(263.2377108872438)),
 (np.float64(1310.7496055346528), np.float64(425.0519480519481)),
 (np.float64(1312.699174657119), np.float64(446.49720839907764)),
 (np.float64(1310.7496055346528), np.float64(610.2610146862485)),
 (np.float64(1312.699174657119), np.float64(629.7567059109116)),
 (np.float64(1310.7496055346528), np.float64(787.6718048306834)),
 (np.float64(1310.7496055346528), np.float64(809.117065177813)),
 (np.float64(1310.7496055346528), np.float64(968.9817332200511)),
 (np.float64(1308.8000364121865), np.float64(988.4774244447142)),
 (np.float64(1310.7496055346528), np.float64(1148.3420924869524)),
 (np.float64(739.5258526520211), np.float64(659.0002427479064))]
im2_pts = [(np.float64(1092.3978638184249), np.float64(526.4295424201968)),
 (np.float64(1236.6659788809325), np.float64(528.379111542663)),
 (np.float64(1086.549156451026), np.float64(725.2855929117612)),
 (np.float64(1234.7164097584662), np.float64(729.1847311566938)),
 (np.float64(950.0793178783836), np.float64(261.28814176477727)),
 (np.float64(948.1297487559173), np.float64(417.2536715620829)),
 (np.float64(948.1297487559173), np.float64(436.74936278674613)),
 (np.float64(946.180179633451), np.float64(586.8661852166526)),
 (np.float64(944.2306105109847), np.float64(608.3114455637821)),
 (np.float64(942.2810413885184), np.float64(756.4786988712224)),
 (np.float64(940.3314722660521), np.float64(775.9743900958855)),
 (np.float64(944.2306105109847), np.float64(924.1416434033257)),
 (np.float64(940.3314722660521), np.float64(943.6373346279889)),
 (np.float64(940.3314722660521), np.float64(1093.7541570578956)),
 (np.float64(365.20858113848783), np.float64(629.7567059109116))]

Firstly, project im1 onto the pp of im2

In [439]:
im1_pts = get_image_points(im1_float)

In [440]:
im1_pts

[(np.float64(606.9551523243115), np.float64(222.29675931545103)),
 (np.float64(571.8629081199176), np.float64(664.8489501153053)),
 (np.float64(638.1482582837725), np.float64(1169.787352834082)),
 (np.float64(866.247845612332), np.float64(594.6644617065178)),
 (np.float64(813.6094793057412), np.float64(653.1515353805074)),
 (np.float64(1135.288384512684), np.float64(167.70882388639416)),
 (np.float64(1182.0780434518756), np.float64(727.2351620342275)),
 (np.float64(1256.1616701055957), np.float64(818.8649107901446)),
 (np.float64(1675.3190314358544), np.float64(579.0679087267873)),
 (np.float64(1599.285835659668), np.float64(649.2523971355748))]

In [441]:
im2_pts = get_image_points(im2_float)

In [442]:
im2_pts

[(np.float64(141.0081320548611), np.float64(156.01140915159613)),
 (np.float64(94.21847311566944), np.float64(662.899380992839)),
 (np.float64(195.59606748391798), np.float64(1228.2744265080714)),
 (np.float64(458.78789901687105), np.float64(582.9670469717199)),
 (np.float64(396.40168709794887), np.float64(643.4036897681758)),
 (np.float64(729.7780070396896), np.float64(165.75925476392786)),
 (np.float64(786.3155115912127), np.float64(711.638609054497)),
 (np.float64(852.6008617550676), np.float64(797.419650443015)),
 (np.float64(1187.9267508192745), np.float64(577.118339604321)),
 (np.float64(1129.439677145285), np.float64(635.6054132783106))]

In [443]:
H12 = computeH(im1_pts, im2_pts)

In [444]:
im1_projected, im1_dx, im1_dy = warp_image_helper_nn(im1_float, H12)

1822 2083
1822 2083
1822 2083


In [413]:
skio.imshow(im1_projected)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/1848739466.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(im1_projected)


Secondly, project im3 onto the pp of im2

In [446]:
im3_pts = get_image_points(im3_float)

In [447]:
im3_pts

[(np.float64(41.58010680907876), np.float64(117.02002670226989)),
 (np.float64(53.277521543876674), np.float64(592.7148925840515)),
 (np.float64(53.277521543876674), np.float64(662.899380992839)),
 (np.float64(684.9379172229642), np.float64(196.9523607233889)),
 (np.float64(445.14091515960695), np.float64(736.9830076465591)),
 (np.float64(519.224541813327), np.float64(824.7136181575435)),
 (np.float64(858.4495691224665), np.float64(600.5131690739169)),
 (np.float64(805.811202815876), np.float64(657.05067362544)),
 (np.float64(1158.6832139822798), np.float64(255.43943439737836)),
 (np.float64(1197.6745964316062), np.float64(1058.6619128535017))]

In [453]:
im2_pts = get_image_points(im2_float)

In [454]:
im2_pts

[(np.float64(441.24177691467423), np.float64(159.91054739652873)),
 (np.float64(456.83832989440475), np.float64(582.9670469717199)),
 (np.float64(458.78789901687105), np.float64(643.4036897681758)),
 (np.float64(1008.5663915523733), np.float64(171.60796213132676)),
 (np.float64(786.3155115912127), np.float64(709.6890399320307)),
 (np.float64(848.701723510135), np.float64(795.4700813205487)),
 (np.float64(1187.9267508192745), np.float64(575.1687704818547)),
 (np.float64(1129.439677145285), np.float64(633.6558441558442)),
 (np.float64(1536.899623740746), np.float64(183.30537686612456)),
 (np.float64(1581.739713557471), np.float64(1080.1071732006312))]

In [455]:
H32 = computeH(im3_pts, im2_pts)

In [456]:
im3_projected, im3_dx, im3_dy = warp_image_helper_nn(im3_float, H32)

1656 1938
1656 1938
1656 1938


In [157]:
skio.imshow(im3_projected)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/358993683.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(im3_projected)


Now, put all images on the same mosaic (mosaic with same size), as well as their respective masks

In [457]:
im1_x0, im1_y0 = im1_dx, im1_dy
im1_x1, im1_y1 = im1_dx + im1_projected.shape[0], im1_dy + im1_projected.shape[1]

im2_x0, im2_y0 = 0, 0
im2_x1, im2_y1 = im2_float.shape[0], im2_float.shape[1]

im3_x0, im3_y0 = im3_dx, im3_dy
im3_x1, im3_y1 = im3_dx + im3_projected.shape[0], im3_dy + im3_projected.shape[1]

In [458]:
overall_x_init = min(im1_x0, im2_x0, im3_x0)
overall_x_end = max(im1_x1, im2_x1, im3_x1)
overall_y_init = min(im1_y0, im2_y0, im3_y0)
overall_y_end = max(im1_y1, im2_y1, im3_y1)
width, height = overall_y_end - overall_y_init, overall_x_end - overall_x_init

In [459]:
image1_mask_small, _, _ = get_binary_mask(im1_float, H12)
res_image1 = [[np.array([0.0, 0.0, 0.0]) for _ in range(width)] for _ in range(height)]
mask_image1 = [[0.0 for _ in range(width)] for _ in range(height)]
for row in range(im1_projected.shape[0]):
    for col in range(im1_projected.shape[1]):
        im1_row, im1_col = row + im1_x0, col + im1_y0
        mosaic_row, mosaic_col = im1_row - overall_x_init, im1_col - overall_y_init
        res_image1[mosaic_row][mosaic_col] = im1_projected[row][col]
        mask_image1[mosaic_row][mosaic_col] = image1_mask_small[row][col]
res_image1 = np.array(res_image1)
mask_image1 = np.array(mask_image1)
skio.imsave("data/A.1/set5/trials/1_trial.jpg", (res_image1 * 255).astype(np.uint8))

1822 2083


In [460]:
res_image2 = [[np.array([0.0, 0.0, 0.0]) for _ in range(width)] for _ in range(height)]
mask_image2 = [[0.0 for _ in range(width)] for _ in range(height)]
for row in range(im2_float.shape[0]):
    for col in range(im2_float.shape[1]):
        im2_row, im2_col = row + im2_x0, col + im2_y0
        mosaic_row, mosaic_col = im2_row - overall_x_init, im2_col - overall_y_init
        res_image2[mosaic_row][mosaic_col] = im2_float[row][col]
        mask_image2[mosaic_row][mosaic_col] = 1.0
res_image2 = np.array(res_image2)
mask_image2 = np.array(mask_image2)
skio.imsave("data/A.1/set5/trials/2_trial.jpg", (res_image2 * 255).astype(np.uint8))

In [461]:
image3_mask_small, _, _ = get_binary_mask(im3_float, H32)
res_image3 = [[np.array([0.0, 0.0, 0.0]) for _ in range(width)] for _ in range(height)]
mask_image3 = [[0.0 for _ in range(width)] for _ in range(height)]
for row in range(im3_projected.shape[0]):
    for col in range(im3_projected.shape[1]):
        im3_row, im3_col = row + im3_x0, col + im3_y0
        mosaic_row, mosaic_col = im3_row - overall_x_init, im3_col - overall_y_init
        res_image3[mosaic_row][mosaic_col] = im3_projected[row][col]
        mask_image3[mosaic_row][mosaic_col] = image3_mask_small[row][col]
res_image3 = np.array(res_image3)
mask_image3 = np.array(mask_image3)
skio.imsave("data/A.1/set5/trials/3_trial.jpg", (res_image3 * 255).astype(np.uint8))

1656 1938


Blend image1 with image2

In [462]:
d1 = edt(mask_image1)
d2 = edt(mask_image2)

In [463]:
w = d1 / (d1 + d2 + 1e-8)
w = np.clip(w, 0, 1)

In [464]:
w_reshaped = w.reshape(w.shape[0], w.shape[1], 1)

In [465]:
res12 = res_image1 * w_reshaped + res_image2 * (1 - w_reshaped)

In [394]:
skio.imshow(res12)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/3063671951.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(res12)


Blend image12 with image3

In [466]:
d12 = edt(mask_image1 + mask_image2)
d3 = edt(mask_image3)

In [467]:
w = d12 / (d12 + d3 + 1e-8)
w = np.clip(w, 0, 1)

In [468]:
w_reshaped = w.reshape(w.shape[0], w.shape[1], 1)

In [469]:
res123 = res12 * w_reshaped + res_image3 * (1 - w_reshaped)

In [399]:
skio.imshow(res123)

/var/folders/zd/6l9_l17d02d4msw14wpqz0lw0000gn/T/ipykernel_94024/346618293.py:1: FutureWarning: `imshow` is deprecated since version 0.25 and will be removed in version 0.27. Please use `matplotlib`, `napari`, etc. to visualize images.
  skio.imshow(res123)


In [470]:
skio.imsave("data/A.1/set5/res.jpg", (res123 * 255).astype(np.uint8))